In [1]:
import joblib
import numpy as np 
import mne
import numpy as np
from scipy.stats import skew, kurtosis
from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
import pandas as pd
from scipy.fftpack import fft
import pywt  # For wavelet transform
import os

In [2]:
def preprocess_eeg(vhdr_file):
    """
    Preprocess EEG data with comprehensive artifact removal.
    
    This function combines the robust preprocessing pipeline from Method 1 with
    the extra processing steps from Method 2:
      - Additional artifact checks (ECG artifact removal)
      - Channel selection for ICA (only EEG and EOG channels)
      - Explicit reference projection

    Parameters
    ----------
    vhdr_file : str
        Path to BrainVision header file.
    save_dir : str or None
        Directory to save preprocessed data plots (optional).

    Returns
    -------
    raw : mne.io.Raw
        Preprocessed EEG data with artifacts removed.
    """
    # 1. Load the raw data
    raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)    
    # 3. Drop unnecessary channels if they exist
    if 'Mass' in raw.ch_names:
        raw.drop_channels('Mass')
    
    # 4. Set non-EEG channel types if available
    non_eeg_channels = {
        'VPVA': 'eog',
        'VNVB': 'eog',
        'HPHL': 'eog',
        'HNHR': 'eog',
        'Erbs': 'ecg',
        'OrbOcc': 'emg',
    }
    channel_types = {ch: typ for ch, typ in non_eeg_channels.items() if ch in raw.ch_names}
    if channel_types:
        raw.set_channel_types(channel_types)
    
    # 5. Set the montage (use on_missing='ignore' to avoid errors for auxiliary channels)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, on_missing='ignore')
    
    # 6. Apply a notch filter (50 Hz) to EEG channels and bandpass filter (0.1 - 40 Hz)
    raw.notch_filter(freqs=50, picks='eeg')
    raw.filter(l_freq=0.1, h_freq=40.0)
    
    # 7. Set average reference with projection and apply the projection
    raw.set_eeg_reference('average', projection=True)
    raw = raw.apply_proj()
    
    # 8. Instantiate the ICA model (with additional iterations for convergence)
    ica = ICA(n_components=20, random_state=42, max_iter=800)
    
    # 9. Pick only EEG and EOG channels for ICA fitting to reduce computational load
    eeg_picks = mne.pick_types(raw.info, eeg=True, eog=True, exclude='bads')
    ica.fit(raw, picks=eeg_picks)
    
    # --- Extra Processing Steps ---
    # ECG artifact removal: detect and exclude ECG-related ICA components.
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, ch_name='Erbs')
    ica.exclude = ecg_indices  # Start exclusion list with ECG-related components
    
    # EOG artifact removal: detect and add EOG-related ICA components to the exclusion list.
    eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=['VPVA', 'VNVB', 'HPHL', 'HNHR'])
    ica.exclude.extend(eog_indices)

    # 10. Apply ICA to remove the identified artifact components (modifies raw in place)
    raw = ica.apply(raw)
    # print(f"Preprocessing complete.")
    return raw


In [3]:
raw = mne.io.read_raw_brainvision("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr", preload=True)

Extracting parameters from /home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 60014  =      0.000 ...   120.028 secs...


In [5]:
raw = preprocess_eeg("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr")

Extracting parameters from /home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 60014  =      0.000 ...   120.028 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.7s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 136 (average pulse 67.98300424893776 / min.)
Not setting metadata
136 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 136 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 3 ICA components
    Projecting back using 30 PCA components


In [6]:
import os
import mne

def slice_and_save_eeg(vhdr_file_path, output_dir, slice_duration=60):
    """Preprocess, slice, and save EEG data into 60-second segments."""
    
    # Extract the base name for saving
    base_name = os.path.splitext(os.path.basename(vhdr_file_path))[0]
    
    # Perform preprocessing
    raw = preprocess_eeg(vhdr_file_path)
    
    # Get sampling frequency
    sfreq = raw.info['sfreq']
    
    # Calculate the number of samples per slice
    samples_per_slice = int(slice_duration * sfreq)
    
    # Calculate the total number of full slices
    num_slices = len(raw) // samples_per_slice

    # Slice and save full segments
    for i in range(num_slices):
        start_sample = i * samples_per_slice
        end_sample = (i + 1) * samples_per_slice

        # Create a new raw object for the slice
        sliced_raw = raw.copy().crop(
            tmin=start_sample / sfreq, 
            tmax=end_sample / sfreq, 
            include_tmax=False
        )
        
        # Construct the filename for the slice
        slice_filename = f"{base_name}_{i + 1}_eeg.fif"
        
        # Save the slice in the output directory
        slice_output_path = os.path.join(output_dir, slice_filename)
        sliced_raw.save(slice_output_path, overwrite=True)
        print(f"Slice {i + 1} saved as: {slice_output_path}")

In [8]:
slice_and_save_eeg("/home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr","../")

Extracting parameters from /home/admincit/Desktop/ibrains_dataset/dataset_grouped/mdd/sub-87999321/ses-1/eeg/sub-87999321_ses-1_task-restEC_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 60014  =      0.000 ...   120.028 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.7s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 136 (average pulse 67.98300424893776 / min.)
Not setting metadata
136 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 136 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 3 ICA components
    Projecting back using 30 PCA components
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif
[done]
Slice 1 saved as: ../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif
Writing /mnt/data/sa

In [9]:
raw = mne.io.read_raw_fif("../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif", preload=True)

Opening raw data file ../sub-87999321_ses-1_task-restEC_eeg_1_eeg.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 26) active
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.
Reading 0 ... 29999  =      0.000 ...    59.998 secs...


In [ ]:
print("All slices saved successfully.")

In [11]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            
            for file in os.listdir(session_path):
                if file.endswith("_eeg.vhdr"):
                    inp_path = os.path.join(session_path, file)
                    output_filename = f"{file.replace('.vhdr', '.fif')}"
                    # output_filename = output_filename.replace("task-restcombined", "task-rest_combined")
                    output_path = os.path.join(destination_folder, output_filename)
                    slice_and_save_eeg(inp_path,destination_folder)



In [ ]:
process_folder("../../dataset_s/mdd","../../split_fif_new/mdd")

Extracting parameters from ../../dataset_s/mdd/sub-88008681/ses-1/eeg/sub-88008681_ses-1_task-restEO_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 59989  =      0.000 ...   119.978 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.7s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 137 (average pulse 68.51141856976163 / min.)
Not setting metadata
137 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 137 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

...

Number of ECG events detected : 129 (average pulse 64.48065580325903 / min.)
Not setting metadata
129 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 129 events and 501 original time points ...
0 bad epochs dropped
    Applying projection operator with 1 vector (pre-whitener application)
Using EOG channels: VPVA, VNVB, HPHL, HNHR
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 2 ICA components
    Projecting back using 30 PCA components
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_1_eeg.fif
[done]
Slice 1 saved as: ../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_1_eeg.fif
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_2_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_2_eeg.fif
[done]
Slice 2 saved as: ../../split_fif_new/mdd/sub-88021101_ses-1_task-restEO_eeg_2_eeg.fif


In [12]:
process_folder("../../dataset_s/healthy","../../split_fif_new/healthy")

Extracting parameters from ../../dataset_s/healthy/sub-88048729/ses-1/eeg/sub-88048729_ses-1_task-restEO_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 60007  =      0.000 ...   120.014 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (fir

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 1.0s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 125 (average pulse 62.49166777762965 / min.)
Not setting metadata
125 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection ite

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower pa

...

Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 1.1s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 148 (average pulse 73.60376639092884 / min.)
Not setting metadata
148 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 148 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering targ

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 30 PCA components
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_1_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_1_eeg.fif
[done]
Slice 1 saved as: ../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_1_eeg.fif
Writing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_2_eeg.fif
Closing /mnt/data/saikrishna/Team_4/notebooks/old/../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_2_eeg.fif
[done]
Slice 2 saved as: ../../split_fif_new/healthy/sub-87976457_ses-1_task-restEC_eeg_2_eeg.fif


In [ ]:
def extract_channel_features(raw, fmin=0.5, fmax=50):
    # Select only EEG channels
    raw.pick_types(eeg=True)  # This removes non-EEG channels
    data = raw.get_data()
    channel_names = raw.ch_names
    features = {ch: {} for ch in channel_names}

    # Time-domain features
    for i, ch in enumerate(channel_names):
        features[ch]['mean'] = np.mean(data[i])
        features[ch]['variance'] = np.var(data[i])
        features[ch]['skewness'] = skew(data[i])
        features[ch]['kurtosis'] = kurtosis(data[i])
        features[ch]['peak_to_peak'] = np.ptp(data[i])

        # Fourier Transform (FFT)
        fft_values = np.abs(fft(data[i]))
        features[ch]['fft_mean'] = np.mean(fft_values)
        features[ch]['fft_std'] = np.std(fft_values)
        features[ch]['fft_max'] = np.max(fft_values)
        
        # Wavelet Transform (Morlet)
        wavelet = 'cmor1.5-1.0'
        coeffs, _ = pywt.cwt(data[i], scales=np.arange(1, 129), wavelet=wavelet)
        coeffs = np.abs(coeffs)  # Convert complex values to magnitude
        
        features[ch]['wavelet_energy'] = np.sum(np.square(coeffs))

        # Wavelet Transform (DWT) using Daubechies wavelet (db4) #morle
        # coeffs = pywt.wavedec(data[i], 'db4', level=4)
        # features[ch]['wavelet_energy'] = sum(np.sum(np.square(c)) for c in coeffs)
        
        # features[ch]['wavelet_entropy'] = 0  # Initialize wavelet_entropy
        
        # for c in coeffs:
        #     c = c[np.isfinite(c)]
        #     c_norm = c / (np.sum(np.abs(c)) + 1e-10)
        #     features[ch]['wavelet_entropy'] += -np.sum(c_norm * np.log2(c_norm + 1e-10))

    # Frequency-domain features using PSD
    psd = raw.compute_psd(method='welch', fmin=fmin, fmax=fmax, n_fft=2048)
    psd_data = psd.get_data()
    freqs = psd.freqs
    psd_df = pd.DataFrame(psd_data, columns=freqs, index=channel_names)

    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'slow_alpha': (6, 9), 'alpha': (8, 12),
             'beta': (12, 30), 'gamma': (30, 50)}

    for band, (low, high) in bands.items():
        band_power = psd_df.loc[:, (freqs >= low) & (freqs <= high)].mean(axis=1)
        for ch in channel_names:
            features[ch][f'{band}_power'] = band_power[ch]

    # Frontal Alpha Asymmetry (F3-F4)
    if 'F3' in channel_names and 'F4' in channel_names:
        features['F3_F4_alpha_asymmetry'] = features['F4']['alpha_power'] - features['F3']['alpha_power']

    # Convert features to DataFrame
    features_df = pd.DataFrame(features).T

    return features_df

In [ ]:
def process_and_combine(eo_file_path, ec_file_path, output_file):
    all_features = []

    # Process EO file
    raw_eo = mne.io.read_raw_fif(eo_file_path)
    features_eo = extract_channel_features(raw_eo)
    features_eo['condition'] = 'EO'
    all_features.append(features_eo)

    # Process EC file
    raw_ec = mne.io.read_raw_fif(ec_file_path)
    features_ec = extract_channel_features(raw_ec)
    features_ec['condition'] = 'EC'
    all_features.append(features_ec)

    # Combine EO and EC features
    combined_features = pd.concat(all_features, keys=['EO', 'EC'], names=['condition', 'channel'])
    
    # Save combined features to a single CSV file
    combined_features.to_csv(output_file)
    print(f"Features successfully saved to {output_file}")
    # return combined_features

In [ ]:
process_and_combine("split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.csv")

In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    output_filename=""
    eo_path=""
    ec_path=""
    print(os.listdir(source_folder))
    for sub_id in os.listdir(source_folder):
        subject_path = os.path.join(source_folder, sub_id)
        if not os.path.isdir(subject_path):
            continue
        
        for ses_id in os.listdir(subject_path):
            session_path = os.path.join(subject_path, ses_id, "eeg")
            if not os.path.isdir(session_path):
                continue
            print(ses_id)
            for file in os.listdir(session_path):
                if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
                    eo_path = os.path.join(session_path, file)
                elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
                    ec_file=file
                    ec_path = os.path.join(session_path, file)
                
                print(eo_path,ec_path)
                output_filename=ec_file.replace("EC_eeg","_eeg_combined.csv")
                process_and_combine(eo_path,ec_path,output_filename)

                
                
                # output_path = os.path.join(destination_folder, output_filename)
                




In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes EO and EC files for all subjects and sessions, saving the features to CSV files.

    Args:
        source_folder (str): Path to the root folder containing subject EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    eo_path, ec_path = None, None
    ec_file = None  # Initialize ec_file to avoid the UnboundLocalError

    # Iterate through files in the source folder
    for file in os.listdir(source_folder):
        file_path = os.path.join(source_folder, file)

        if file.endswith("EC_eeg_1.fif") or file.endswith("EC_eeg_2.fif"):
            ec_path = file_path
            ec_file = file  # Store EC file name for output filename generation
        elif file.endswith("EO_eeg_1.fif") or file.endswith("EO_eeg_2.fif"):
            eo_path = file_path

        # Process only when both EO and EC files are found
        if eo_path and ec_path and ec_file:
            output_filename = ec_file.replace("EC_eeg", "eeg_combined.csv")
            output_filepath = os.path.join(destination_folder, output_filename)

            print(f"Processing: \n  EO: {eo_path} \n  EC: {ec_path} \n  Output: {output_filepath}")
            process_and_combine(eo_path, ec_path, output_filepath)

            # Reset paths after processing
            eo_path, ec_path, ec_file = None, None, None


In [ ]:
process_folder("dataset_s/mdd","split_fif/mdd")